![rainbow](https://github.com/ancilcleetus/My-Learning-Journey/assets/25684256/839c3524-2a1d-4779-85a0-83c562e1e5e5)

# 1. Introduction

We have seen that Sliding Window Technique is computationally expensive and inefficient due to the very high number of bounding box positions. Hence, in order to speed up the object detection, we have two options:

1. Use a shallow network with less no of parameters $\implies$ CNN model may not perform well
2. Reduce the no of bounding boxes
    - No need to pass bounding boxes with "Background" class (since CNN do not learn anything)
    - No need for duplicate bounding boxes for same object

RCNN (Region CNN) tries to get only the regions of image in which there is a high probability of an object. Thus, RCNN reduces the no of regions that we pass into the CNN.

![rainbow](https://github.com/ancilcleetus/My-Learning-Journey/assets/25684256/839c3524-2a1d-4779-85a0-83c562e1e5e5)

In [ ]:
# Deep Learning as subset of ML

from IPython import display
display.Image("data/images/DL_01_Intro-01-DL-subset-of-ML.jpg")

![rainbow](https://github.com/ancilcleetus/My-Learning-Journey/assets/25684256/839c3524-2a1d-4779-85a0-83c562e1e5e5)